### Experiment:  Feature Impact on Response Time

Comparison of system response time for different permutations

In [1]:
# Common import
from deh.assessment import QASetRetriever
from deh import settings
from deh.eval import generate_experiment_dataset

import pandas as pd
import os
from pathlib import Path

/home/ubuntu/miniconda3/envs/deh_measurement/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Test Configuration

In [3]:
num_samples:int = 10
experiment_folder:str = "../../data/evaluation/speed-experiment/"
qa_data_set_file:str = "../../data/qas/squad_qas.tsv"

# Create experiment folder:
if not os.path.exists(experiment_folder):
    Path(experiment_folder).mkdir(parents=True, exist_ok=True)


#### Sample QA dataset

In [4]:
qa_set = QASetRetriever.get_qasets(
    file_path = qa_data_set_file,
    sample_size= num_samples
)

print(f"{len(qa_set)} questions sampled from QA corpus ({qa_data_set_file})")

10 questions sampled from QA corpus (../../data/qas/squad_qas.tsv)


#### Get Similiarity Scores based on original question

In [22]:
def convert(response) -> pd.DataFrame:
    """Converts retrieved JSON response to Pandas DataFrame"""
    return pd.json_normalize(
        data=response["response"],
        meta=["execution_time"],
    )

def api_endpoint(**kwargs) -> str:
    """Endpoint for context retrieval w/ hyde (h=true,p=0)."""

    kwargs["h"] = True
    kwargs["e"] = True
    kwargs["lp"] = 0
    kwargs["k"] = 20

    query_params = "&".join([f"{key}={kwargs[key]}" for key in kwargs])
    return (
        f"http://{settings.API_ANSWER_ENDPOINT}/answer?{query_params}"
    )

# Collect response:
exp_df = generate_experiment_dataset(qa_set, convert, api_endpoint)

# Store dataframe:
exp_df.to_pickle( f"{experiment_folder}/no_rag.pkl" )
exp_df[0:10]

Processing 1 of 10 question/answer pairs.
Error processing 1 of 10
Expecting value: line 1 column 1 (char 0)
Processing 2 of 10 question/answer pairs.
Error processing 2 of 10
Expecting value: line 1 column 1 (char 0)
Processing 3 of 10 question/answer pairs.
Error processing 3 of 10
Expecting value: line 1 column 1 (char 0)
Processing 4 of 10 question/answer pairs.
Error processing 4 of 10
Expecting value: line 1 column 1 (char 0)
Processing 5 of 10 question/answer pairs.
Error processing 5 of 10
Expecting value: line 1 column 1 (char 0)
Processing 6 of 10 question/answer pairs.
Error processing 6 of 10
Expecting value: line 1 column 1 (char 0)
Processing 7 of 10 question/answer pairs.
Error processing 7 of 10
Expecting value: line 1 column 1 (char 0)
Processing 8 of 10 question/answer pairs.
Error processing 8 of 10
Expecting value: line 1 column 1 (char 0)
Processing 9 of 10 question/answer pairs.
Error processing 9 of 10
Expecting value: line 1 column 1 (char 0)
Processing 10 of 10

AttributeError: 'NoneType' object has no attribute 'to_pickle'

#### Avg Result

In [21]:
exp_df["response_diff"] = pd.to_timedelta(exp_df["execution_time"]).dt.total_seconds()
exp_df["response_diff"].mean()

1.9